# Net migration
Data source: https://zenodo.org/records/7997134   
Paper: https://www.nature.com/articles/s41562-023-01689-4#data-availability


In [1]:
import os
import gc
import fiona
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from pathlib import Path
from osgeo import gdal, osr
from rasterstats import zonal_stats
from rasterio.warp import calculate_default_transform, reproject, Resampling

BASE_DIR = Path('/Users/wenlanzhang/PycharmProjects/Mapineq/src/data-wrangling')
DATA_DIR = Path('/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford')

# Reproject to 3035

In [2]:
# def reproject_raster(src_path, dst_path, dst_crs='EPSG:3035'):
#     with rasterio.open(src_path) as src:
#         transform, width, height = calculate_default_transform(
#             src.crs, dst_crs, src.width, src.height, *src.bounds)
#         kwargs = src.meta.copy()
#         kwargs.update({
#             'crs': dst_crs,
#             'transform': transform,
#             'width': width,
#             'height': height
#         })

#         with rasterio.open(dst_path, 'w', **kwargs) as dst:
#             for i in range(1, src.count + 1):
#                 reproject(
#                     source=rasterio.band(src, i),
#                     destination=rasterio.band(dst, i),
#                     src_transform=src.transform,
#                     src_crs=src.crs,
#                     dst_transform=transform,
#                     dst_crs=dst_crs,
#                     resampling=Resampling.nearest)  # You can choose other resampling methods

# # Usage
# src_raster = DATA_DIR / "Migration/NetMigration/raster_netMgr_2000_2019_annual.tif"
# dst_raster = DATA_DIR / "Migration/NetMigration/raster_netMgr_2000_2019_annual_3035.tif"
# reproject_raster(src_raster, dst_raster)
# print("Reprojection complete.")

# Processing

In [3]:
raster_fp = DATA_DIR / "Migration/NetMigration/raster_netMgr_2000_2019_annual_3035.tif"
years = list(range(2000, 2020))  # Raster bands correspond to these years

# List of vector files and their corresponding geo_source labels
nuts_years = [2003, 2006, 2010, 2013, 2016, 2021, 2024]
# nuts_years = [2003, 2006]
vector_files = [DATA_DIR / f"NUTS/NUTS_RG_01M_{year}_3035.geojson" for year in nuts_years]

results_list = []

with rasterio.open(raster_fp) as src:
    # Outer tqdm for vector files
    for vector_fp, nuts_year in tqdm(zip(vector_files, nuts_years), total=len(nuts_years), desc="Processing NUTS vectors"):
        gdf = gpd.read_file(vector_fp)
        gdf_3035 = gdf.to_crs(epsg=3035)

        id_cols = ['NUTS_ID']
        gdf_ids = gdf_3035[id_cols].reset_index(drop=True)

        # Inner tqdm for bands/years
        for band_index, year in enumerate(tqdm(years, desc=f"Processing years for NUTS{nuts_year}", leave=False), start=1):
            stats = zonal_stats(
                gdf_3035.geometry,
                raster_fp,
                stats=['mean', 'sum', 'max', 'min', 'median', 'std'],
                band=band_index,
                geojson_out=False
            )

            year_df = pd.DataFrame(stats)
            year_df['obsTime'] = year
            year_df[id_cols] = gdf_ids

            df_long = year_df.melt(
                id_vars=id_cols + ['obsTime'],
                var_name='metric',
                value_name='obsValue'
            )

            # Add the geo_source column
            df_long['geo_source'] = f"NUTS{nuts_year}"

            results_list.append(df_long)

# Combine all results
df_melt = pd.concat(results_list, ignore_index=True)

# Rename and reorder columns
df_melt = df_melt.rename(columns={'NUTS_ID': 'geo'})
df_melt = df_melt[['geo', 'obsTime', 'metric', 'obsValue', 'geo_source']]

df_melt

Processing NUTS vectors: 100%|███████████████████| 7/7 [37:12<00:00, 318.98s/it]


,geo,obsTime,metric,obsValue,geo_source
0,NL11,2000,min,-1211.751465,NUTS2003
1,IE01,2000,min,-231.577240,NUTS2003
2,NL12,2000,min,-1217.874512,NUTS2003
3,DE93,2000,min,-3657.027832,NUTS2003
4,NL13,2000,min,-2188.597900,NUTS2003
...,...,...,...,...,...
1622035,RO,2019,median,-12.950260,NUTS2024
1622036,NO,2019,median,-0.740463,NUTS2024
1622037,PL,2019,median,-2.050199,NUTS2024
1622038,PT,2019,median,-4.778418,NUTS2024


In [4]:
df_melt.to_csv(DATA_DIR/"Migration/Output/Net_migration_20year.csv", index=True, index_label="id")